In [8]:
import pymysql
import pandas as pd
import numpy as np
import scipy.stats as stats

# Configuración de la conexión
endpoint = "mat-trn-src.co4vks4ks8wo.eu-west-1.rds.amazonaws.com"
port = 3306
database = "training"
username = "srcuser"
password = "matillion"

# Conectar a la base de datos y cargar los datos en un DataFrame
try:
    # Crear la conexión a la base de datos
    connection = pymysql.connect(
        host=endpoint,
        user=username,
        password=password,
        database=database,
        port=port
    )
    
    # Crear un cursor para ejecutar la consulta
    with connection.cursor() as cursor:
        sql_query = """
        SELECT `iata`, `airport`, `city`, `state`, `country`, `lat`, `long`, `num_carriers`, `is_active`
        FROM training_rds_airports;
        """
        cursor.execute(sql_query)
        
        # Obtener los resultados
        results = cursor.fetchall()
        
        # Crear un DataFrame a partir de los resultados
        columns = ['iata', 'airport', 'city', 'state', 'country', 'lat', 'long', 'num_carriers', 'is_active']
        df = pd.DataFrame(results, columns=columns)
        
        # Paso 1: Imprimir el DataFrame original
        print("DataFrame original:")
        print(df)
        
        # Paso 2: Transformar la columna 'is_active' de binario a booleano
        df['is_active'] = df['is_active'].apply(lambda x: x == b'\x01')
        print("\nDataFrame después de la transformación de 'is_active':")
        print(df)
        
        # Análisis de cardinalidad para las variables categóricas
        categorical_columns = ['iata', 'airport', 'city', 'state', 'country']
        print("\nCardinalidad de las variables categóricas en el DataFrame original:")
        for column in categorical_columns:
            unique_values = df[column].nunique()
            print(f"{column}: {unique_values} categorías únicas")

        # Verificar normalidad de la columna 'num_carriers'
        print("\nAnálisis de normalidad para la columna 'num_carriers':")

        # Prueba de Shapiro-Wilk
        shapiro_stat, shapiro_p_value = stats.shapiro(df['num_carriers'])
        print(f"Prueba de Shapiro-Wilk: Estadístico={shapiro_stat}, p-valor={shapiro_p_value}")
        
        # Prueba de Kolmogorov-Smirnov
        ks_stat, ks_p_value = stats.kstest(df['num_carriers'], 'norm', args=(df['num_carriers'].mean(), df['num_carriers'].std()))
        print(f"Prueba de Kolmogorov-Smirnov: Estadístico={ks_stat}, p-valor={ks_p_value}")

        # Interpretación de resultados
        alpha = 0.05  # Nivel de significancia
        if shapiro_p_value > alpha and ks_p_value > alpha:
            print("La columna 'num_carriers' sigue una distribución normal (no se rechaza la hipótesis de normalidad).")
        else:
            print("La columna 'num_carriers' no sigue una distribución normal (se rechaza la hipótesis de normalidad).")

        # Paso 3: Aplicar One-Hot Encoding a las variables categóricas
        df_one_hot = pd.get_dummies(df, columns=categorical_columns)
        print("\nDataFrame después de aplicar One-Hot Encoding:")
        print(df_one_hot)
        print(f"\nNúmero de columnas después de One-Hot Encoding: {df_one_hot.shape[1]}")
        
        # Paso 4: Aplicar Frequency Encoding a las mismas variables categóricas
        df_freq = df.copy()  # Crear una copia para no sobrescribir el original
        for column in categorical_columns:
            freq_encoding = df_freq[column].value_counts() / len(df_freq)
            df_freq[column + '_freq'] = df_freq[column].map(freq_encoding)

        # Eliminar las columnas categóricas originales después de Frequency Encoding
        df_freq = df_freq.drop(columns=categorical_columns)
        print("\nDataFrame después de aplicar Frequency Encoding:")
        print(df_freq)
        print(f"\nNúmero de columnas después de Frequency Encoding: {df_freq.shape[1]}")

        # Paso 5: Aplicar Target Encoding a las variables categóricas
        # Agregar una columna de target simulada para el ejemplo de Target Encoding
        np.random.seed(0)
        df['target_value'] = np.random.randint(0, 100, size=len(df))
        print("\nDataFrame con columna de target simulada:")
        print(df)
        
        df_target = df.copy()  # Crear una copia para aplicar Target Encoding
        for column in categorical_columns:
            target_mean = df_target.groupby(column)['target_value'].mean()
            df_target[column + '_target'] = df_target[column].map(target_mean)

        # Eliminar las columnas categóricas originales después de Target Encoding
        df_target = df_target.drop(columns=categorical_columns)
        print("\nDataFrame después de aplicar Target Encoding:")
        print(df_target)
        print(f"\nNúmero de columnas después de Target Encoding: {df_target.shape[1]}")

except pymysql.MySQLError as e:
    print(f"Error al conectar o ejecutar la consulta: {e}")
finally:
    # Cerrar la conexión
    if connection:
        connection.close()


DataFrame original:
     iata                    airport          city state country          lat  \
0     00R       Livingston Municipal    Livingston    TX     USA  30.68586111   
1     01G               Perry-Warsaw         Perry    NY     USA  42.74134667   
2     01M          Tishomingo County       Belmont    MS     USA  34.49166667   
3     02C                    Capitol    Brookfield    WI     USA  43.08751000   
4     03D           Memphis Memorial       Memphis    MO     USA  40.44725889   
...   ...                        ...           ...   ...     ...          ...   
3367  Z55                Lake Louise   Lake Louise    AK     USA  62.29368944   
3368  Z84                      Clear  Clear A.F.B.    AK     USA  64.30120361   
3369  Z95                    Cibecue       Cibecue    AZ     USA  34.00333333   
3370  ZER  Schuylkill Cty/Joe Zerbey    Pottsville    PA     USA  40.70644889   
3371  ZUN                 Black Rock          Zuni    NM     USA  35.08322694   

       